# Multi-Agent Mechanic Business Bio Creator

This instructional notebook demonstrates how to build a multi-agent AI application using AWS Bedrock that:
1. Takes mechanic business information as input
2. Uses multiple specialized agents to process the information
3. Returns an edited street view image and generated bio

## Architecture Overview
- **Agent 1**: Ingests mechanic name and address
- **Agent 2**: Uses Google Maps MCP to find the business
- **Agent 3**: Retrieves street view image
- **Agent 4**: Edits image to remove phone numbers
- **Agent 5**: Searches web for business information
- **Agent 6**: Writes business bio
- **Agent 7**: Reviews and refines the bio
- **Frontend**: Gradio interface for user interaction

## 1. Setup and Dependencies

In [ ]:
# Install required packages
!pip install boto3 gradio pillow requests python-dotenv googlemaps beautifulsoup4 langchain langchain-aws

In [ ]:
import os
import json
import boto3
import gradio as gr
import requests
from PIL import Image
from io import BytesIO
import base64
from typing import Dict, Any, Tuple, Optional
from datetime import datetime
import logging
from dotenv import load_dotenv
import time
from dataclasses import dataclass
from enum import Enum

# Load environment variables
load_dotenv()

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

## 2. AWS Configuration

In [ ]:
# AWS Configuration
AWS_REGION = os.getenv('AWS_REGION', 'us-east-1')
GOOGLE_MAPS_API_KEY = os.getenv('GOOGLE_MAPS_API_KEY', '')

# Initialize AWS clients
bedrock_runtime = boto3.client(
    service_name='bedrock-runtime',
    region_name=AWS_REGION
)

bedrock_agent_runtime = boto3.client(
    service_name='bedrock-agent-runtime',
    region_name=AWS_REGION
)

# Model configuration
MODEL_ID = "anthropic.claude-3-sonnet-20240229-v1:0"

## 3. Data Models

In [ ]:
@dataclass
class MechanicInfo:
    """Data model for mechanic information"""
    name: str
    address: str
    place_id: Optional[str] = None
    latitude: Optional[float] = None
    longitude: Optional[float] = None
    
@dataclass
class ProcessingResult:
    """Result from the multi-agent processing"""
    edited_image: Optional[Image.Image]
    bio: str
    metadata: Dict[str, Any]

class AgentRole(Enum):
    """Enum for different agent roles"""
    INGESTION = "ingestion_agent"
    MAPS_FINDER = "maps_finder_agent"
    IMAGE_RETRIEVER = "image_retriever_agent"
    IMAGE_EDITOR = "image_editor_agent"
    WEB_SEARCHER = "web_searcher_agent"
    BIO_WRITER = "bio_writer_agent"
    BIO_REVIEWER = "bio_reviewer_agent"

## 4. Base Agent Class

In [ ]:
class BaseAgent:
    """Base class for all agents in the system"""
    
    def __init__(self, role: AgentRole, model_id: str = MODEL_ID):
        self.role = role
        self.model_id = model_id
        self.logger = logging.getLogger(f"{self.__class__.__name__}")
        
    def invoke_bedrock(self, prompt: str, system_prompt: str = "") -> str:
        """Invoke Bedrock model with given prompt"""
        try:
            messages = [{"role": "user", "content": prompt}]
            
            request_body = {
                "anthropic_version": "bedrock-2023-05-31",
                "max_tokens": 2000,
                "messages": messages,
                "temperature": 0.7
            }
            
            if system_prompt:
                request_body["system"] = system_prompt
            
            response = bedrock_runtime.invoke_model(
                modelId=self.model_id,
                contentType="application/json",
                accept="application/json",
                body=json.dumps(request_body)
            )
            
            response_body = json.loads(response['body'].read())
            return response_body['content'][0]['text']
            
        except Exception as e:
            self.logger.error(f"Error invoking Bedrock: {str(e)}")
            raise
    
    def process(self, input_data: Dict[str, Any]) -> Dict[str, Any]:
        """Process method to be implemented by each agent"""
        raise NotImplementedError("Each agent must implement the process method")

## 5. Agent 1: Ingestion Agent

In [ ]:
class IngestionAgent(BaseAgent):
    """Agent responsible for ingesting and validating mechanic information"""
    
    def __init__(self):
        super().__init__(AgentRole.INGESTION)
        
    def process(self, input_data: Dict[str, Any]) -> Dict[str, Any]:
        """Process and validate mechanic information"""
        self.logger.info(f"Ingesting mechanic data: {input_data}")
        
        name = input_data.get('name', '').strip()
        address = input_data.get('address', '').strip()
        
        if not name or not address:
            raise ValueError("Both name and address are required")
        
        # Use Bedrock to enhance/validate the address
        system_prompt = "You are an address validation agent. Format and validate the given address."
        prompt = f"""Validate and format this mechanic shop information:
        Name: {name}
        Address: {address}
        
        Return a JSON with 'name' and 'formatted_address' fields."""
        
        response = self.invoke_bedrock(prompt, system_prompt)
        
        try:
            # Parse JSON from response
            import re
            json_match = re.search(r'\{.*\}', response, re.DOTALL)
            if json_match:
                validated_data = json.loads(json_match.group())
            else:
                validated_data = {"name": name, "formatted_address": address}
        except:
            validated_data = {"name": name, "formatted_address": address}
        
        return {
            "mechanic_info": MechanicInfo(
                name=validated_data.get('name', name),
                address=validated_data.get('formatted_address', address)
            ),
            "timestamp": datetime.now().isoformat()
        }

## 6. Agent 2: Google Maps Finder Agent

In [ ]:
class MapsFinderAgent(BaseAgent):
    """Agent that uses Google Maps API to find business location"""
    
    def __init__(self):
        super().__init__(AgentRole.MAPS_FINDER)
        self.maps_api_key = GOOGLE_MAPS_API_KEY
        
    def process(self, input_data: Dict[str, Any]) -> Dict[str, Any]:
        """Find business using Google Maps API (simulated)"""
        mechanic_info = input_data.get('mechanic_info')
        
        if not mechanic_info:
            raise ValueError("Mechanic info not found in input data")
        
        self.logger.info(f"Searching for: {mechanic_info.name} at {mechanic_info.address}")
        
        # Simulate Google Maps API call
        # In production, you would use actual Google Maps API
        search_query = f"{mechanic_info.name} {mechanic_info.address}"
        
        # Mock coordinates (in production, get from Google Maps API)
        # Example coordinates for demonstration
        mock_lat = 40.7128
        mock_lng = -74.0060
        
        mechanic_info.latitude = mock_lat
        mechanic_info.longitude = mock_lng
        mechanic_info.place_id = "mock_place_id_123"
        
        return {
            **input_data,
            "location_found": True,
            "coordinates": {"lat": mock_lat, "lng": mock_lng}
        }

## 7. Agent 3: Street View Image Retriever

In [ ]:
class ImageRetrieverAgent(BaseAgent):
    """Agent that retrieves street view image of the business"""
    
    def __init__(self):
        super().__init__(AgentRole.IMAGE_RETRIEVER)
        
    def process(self, input_data: Dict[str, Any]) -> Dict[str, Any]:
        """Retrieve street view image"""
        mechanic_info = input_data.get('mechanic_info')
        coordinates = input_data.get('coordinates')
        
        if not coordinates:
            raise ValueError("Coordinates not found")
        
        self.logger.info(f"Retrieving street view for coordinates: {coordinates}")
        
        # Create a placeholder image for demonstration
        # In production, you would use Google Street View API
        placeholder_image = Image.new('RGB', (640, 480), color='lightgray')
        
        # Draw some text on the image
        from PIL import ImageDraw, ImageFont
        draw = ImageDraw.Draw(placeholder_image)
        text = f"{mechanic_info.name}\n{mechanic_info.address}\n(Street View Placeholder)"
        
        # Simple text placement
        draw.text((50, 200), text, fill='black')
        
        # Add mock phone number for demonstration
        draw.text((50, 300), "Call: 555-1234", fill='blue')
        
        return {
            **input_data,
            "street_view_image": placeholder_image
        }

## 8. Agent 4: Image Editor Agent

In [ ]:
class ImageEditorAgent(BaseAgent):
    """Agent that edits images to remove phone numbers"""
    
    def __init__(self):
        super().__init__(AgentRole.IMAGE_EDITOR)
        
    def process(self, input_data: Dict[str, Any]) -> Dict[str, Any]:
        """Edit image to remove phone numbers"""
        image = input_data.get('street_view_image')
        
        if not image:
            raise ValueError("No image found to edit")
        
        self.logger.info("Editing image to remove phone numbers")
        
        # Simulate image editing (in production, use actual image processing)
        edited_image = image.copy()
        
        # Mock editing - blur area where phone number would be
        from PIL import ImageFilter, ImageDraw
        
        # Create a mask for the phone number area
        mask = Image.new('L', edited_image.size, 0)
        draw = ImageDraw.Draw(mask)
        # Draw rectangle over phone number area
        draw.rectangle([45, 295, 200, 320], fill=255)
        
        # Apply blur to that area
        blurred = edited_image.filter(ImageFilter.GaussianBlur(radius=10))
        edited_image = Image.composite(blurred, edited_image, mask)
        
        # Add watermark
        draw = ImageDraw.Draw(edited_image)
        draw.text((10, 10), "Edited by AI", fill='red')
        
        return {
            **input_data,
            "edited_image": edited_image,
            "editing_complete": True
        }

## 9. Agent 5: Web Search Agent

In [ ]:
class WebSearchAgent(BaseAgent):
    """Agent that searches the web for business information"""
    
    def __init__(self):
        super().__init__(AgentRole.WEB_SEARCHER)
        
    def process(self, input_data: Dict[str, Any]) -> Dict[str, Any]:
        """Search web for business information"""
        mechanic_info = input_data.get('mechanic_info')
        
        self.logger.info(f"Searching web for: {mechanic_info.name}")
        
        # Use Bedrock to simulate web search results
        system_prompt = """You are a web search agent. Generate realistic search results 
        for an auto mechanic business including services, reviews, and history."""
        
        prompt = f"""Generate web search results for this mechanic business:
        Name: {mechanic_info.name}
        Address: {mechanic_info.address}
        
        Include: services offered, years in business, specializations, customer reviews summary.
        Format as JSON."""
        
        response = self.invoke_bedrock(prompt, system_prompt)
        
        # Parse response
        try:
            import re
            json_match = re.search(r'\{.*\}', response, re.DOTALL)
            if json_match:
                search_results = json.loads(json_match.group())
            else:
                search_results = {
                    "services": ["General Repairs", "Oil Changes", "Brake Service"],
                    "years_in_business": "10+",
                    "specialization": "Domestic and Import Vehicles",
                    "rating": "4.5/5"
                }
        except:
            search_results = {
                "services": ["General Repairs", "Oil Changes", "Brake Service"],
                "years_in_business": "10+",
                "specialization": "Domestic and Import Vehicles",
                "rating": "4.5/5"
            }
        
        return {
            **input_data,
            "web_search_results": search_results
        }

## 10. Agent 6: Bio Writer Agent

In [ ]:
class BioWriterAgent(BaseAgent):
    """Agent that writes a bio for the business"""
    
    def __init__(self):
        super().__init__(AgentRole.BIO_WRITER)
        
    def process(self, input_data: Dict[str, Any]) -> Dict[str, Any]:
        """Write a bio based on collected information"""
        mechanic_info = input_data.get('mechanic_info')
        search_results = input_data.get('web_search_results', {})
        
        self.logger.info(f"Writing bio for: {mechanic_info.name}")
        
        system_prompt = """You are a professional business bio writer. 
        Write engaging, informative bios for auto mechanic businesses."""
        
        prompt = f"""Write a professional bio for this mechanic business:
        
        Business Name: {mechanic_info.name}
        Address: {mechanic_info.address}
        
        Additional Information:
        {json.dumps(search_results, indent=2)}
        
        Write a 200-300 word bio that highlights:
        - The business's history and experience
        - Services offered
        - What makes them unique
        - Community involvement
        - Customer satisfaction
        
        Make it professional but friendly."""
        
        bio = self.invoke_bedrock(prompt, system_prompt)
        
        return {
            **input_data,
            "draft_bio": bio
        }

## 11. Agent 7: Bio Review Agent

In [ ]:
class BioReviewAgent(BaseAgent):
    """Agent that reviews and refines the bio"""
    
    def __init__(self):
        super().__init__(AgentRole.BIO_REVIEWER)
        
    def process(self, input_data: Dict[str, Any]) -> Dict[str, Any]:
        """Review and refine the bio"""
        draft_bio = input_data.get('draft_bio', '')
        mechanic_info = input_data.get('mechanic_info')
        
        self.logger.info("Reviewing and refining bio")
        
        system_prompt = """You are an expert editor. Review and refine business bios for 
        clarity, accuracy, and engagement. Fix any issues and enhance the content."""
        
        prompt = f"""Review and refine this mechanic business bio:
        
        Original Bio:
        {draft_bio}
        
        Requirements:
        - Ensure accuracy and professionalism
        - Check grammar and flow
        - Make it engaging but not overly promotional
        - Keep it between 200-300 words
        - Ensure it mentions the business name: {mechanic_info.name}
        
        Provide the refined bio."""
        
        final_bio = self.invoke_bedrock(prompt, system_prompt)
        
        return {
            **input_data,
            "final_bio": final_bio,
            "review_complete": True
        }

## 12. Orchestrator

In [ ]:
class MultiAgentOrchestrator:
    """Orchestrates the multi-agent workflow"""
    
    def __init__(self):
        self.agents = {
            AgentRole.INGESTION: IngestionAgent(),
            AgentRole.MAPS_FINDER: MapsFinderAgent(),
            AgentRole.IMAGE_RETRIEVER: ImageRetrieverAgent(),
            AgentRole.IMAGE_EDITOR: ImageEditorAgent(),
            AgentRole.WEB_SEARCHER: WebSearchAgent(),
            AgentRole.BIO_WRITER: BioWriterAgent(),
            AgentRole.BIO_REVIEWER: BioReviewAgent()
        }
        self.logger = logging.getLogger("Orchestrator")
        
    def process_mechanic(self, name: str, address: str) -> ProcessingResult:
        """Process mechanic information through all agents"""
        
        self.logger.info(f"Starting multi-agent processing for {name}")
        
        # Initial data
        data = {"name": name, "address": address}
        
        # Execute agents in sequence
        agent_sequence = [
            AgentRole.INGESTION,
            AgentRole.MAPS_FINDER,
            AgentRole.IMAGE_RETRIEVER,
            AgentRole.IMAGE_EDITOR,
            AgentRole.WEB_SEARCHER,
            AgentRole.BIO_WRITER,
            AgentRole.BIO_REVIEWER
        ]
        
        for role in agent_sequence:
            try:
                self.logger.info(f"Executing {role.value}")
                agent = self.agents[role]
                data = agent.process(data)
                time.sleep(0.5)  # Simulate processing time
            except Exception as e:
                self.logger.error(f"Error in {role.value}: {str(e)}")
                raise
        
        # Prepare final result
        result = ProcessingResult(
            edited_image=data.get('edited_image'),
            bio=data.get('final_bio', ''),
            metadata={
                "processing_time": datetime.now().isoformat(),
                "mechanic_info": data.get('mechanic_info').__dict__ if data.get('mechanic_info') else {},
                "agents_executed": [role.value for role in agent_sequence]
            }
        )
        
        return result

## 13. Gradio Interface

In [ ]:
def create_gradio_interface():
    """Create the Gradio interface for the application"""
    
    orchestrator = MultiAgentOrchestrator()
    
    def process_mechanic_info(name: str, address: str, progress=gr.Progress()):
        """Process mechanic information and return results"""
        
        if not name or not address:
            return None, "Please provide both mechanic name and address", ""
        
        try:
            # Update progress
            progress(0.1, desc="Starting multi-agent processing...")
            
            # Process through agents
            progress(0.3, desc="Finding business location...")
            progress(0.5, desc="Retrieving and editing image...")
            progress(0.7, desc="Searching web and writing bio...")
            
            result = orchestrator.process_mechanic(name, address)
            
            progress(0.9, desc="Finalizing results...")
            
            # Format metadata for display
            metadata_str = json.dumps(result.metadata, indent=2)
            
            progress(1.0, desc="Complete!")
            
            return result.edited_image, result.bio, metadata_str
            
        except Exception as e:
            logger.error(f"Error processing: {str(e)}")
            return None, f"Error: {str(e)}", ""
    
    # Create Gradio interface
    with gr.Blocks(title="Mechanic Bio Creator", theme=gr.themes.Soft()) as demo:
        gr.Markdown(
            """
            # 🔧 Multi-Agent Mechanic Bio Creator
            
            This application uses multiple AWS Bedrock agents to:
            1. Find mechanic businesses using Google Maps
            2. Retrieve and edit street view images
            3. Generate professional business bios
            
            Enter a mechanic shop name and address to get started!
            """
        )
        
        with gr.Row():
            with gr.Column(scale=1):
                gr.Markdown("### Input Information")
                name_input = gr.Textbox(
                    label="Mechanic Shop Name",
                    placeholder="e.g., Joe's Auto Repair",
                    lines=1
                )
                address_input = gr.Textbox(
                    label="Address",
                    placeholder="e.g., 123 Main St, Springfield, IL 62701",
                    lines=2
                )
                
                process_btn = gr.Button("🚀 Process", variant="primary", size="lg")
                
                gr.Markdown(
                    """
                    ### How it works:
                    - **7 Specialized Agents** work together
                    - **AWS Bedrock** powers the AI
                    - **Google Maps Integration** for location data
                    - **Automated Image Editing** for privacy
                    """
                )
                
            with gr.Column(scale=2):
                gr.Markdown("### Results")
                
                with gr.Tab("Edited Image"):
                    image_output = gr.Image(
                        label="Street View (Phone Numbers Removed)",
                        type="pil"
                    )
                
                with gr.Tab("Business Bio"):
                    bio_output = gr.Textbox(
                        label="Generated Bio",
                        lines=10,
                        max_lines=15
                    )
                
                with gr.Tab("Processing Metadata"):
                    metadata_output = gr.Textbox(
                        label="Agent Processing Details",
                        lines=10,
                        max_lines=20
                    )
        
        # Examples
        gr.Examples(
            examples=[
                ["Mike's Auto Service", "456 Oak Avenue, Chicago, IL 60601"],
                ["Quick Fix Automotive", "789 Elm Street, Austin, TX 78701"],
                ["Premier Car Care Center", "321 Pine Road, Seattle, WA 98101"]
            ],
            inputs=[name_input, address_input]
        )
        
        # Event handlers
        process_btn.click(
            fn=process_mechanic_info,
            inputs=[name_input, address_input],
            outputs=[image_output, bio_output, metadata_output]
        )
        
    return demo

# Create the interface
demo = create_gradio_interface()

## 14. Launch the Application

In [ ]:
# Launch the Gradio interface
if __name__ == "__main__":
    demo.launch(
        share=False,  # Set to True to create a public link
        server_name="127.0.0.1",
        server_port=7860,
        show_error=True
    )

## 15. Testing and Validation

In [ ]:
# Test individual agents
def test_agents():
    """Test individual agent functionality"""
    
    print("Testing Individual Agents...\n")
    
    # Test Ingestion Agent
    print("1. Testing Ingestion Agent")
    ingestion_agent = IngestionAgent()
    test_data = {"name": "Test Auto Shop", "address": "123 Test St"}
    result = ingestion_agent.process(test_data)
    print(f"   Result: {result['mechanic_info'].name}\n")
    
    # Test Maps Finder Agent
    print("2. Testing Maps Finder Agent")
    maps_agent = MapsFinderAgent()
    result = maps_agent.process(result)
    print(f"   Location found: {result['location_found']}\n")
    
    # Test Image Retriever Agent
    print("3. Testing Image Retriever Agent")
    image_agent = ImageRetrieverAgent()
    result = image_agent.process(result)
    print(f"   Image retrieved: {result['street_view_image'] is not None}\n")
    
    print("All agents tested successfully!")

# Run tests
# test_agents()

## 16. Configuration and Environment Variables

Create a `.env` file with the following variables:

```
AWS_REGION=us-east-1
AWS_ACCESS_KEY_ID=your_access_key
AWS_SECRET_ACCESS_KEY=your_secret_key
GOOGLE_MAPS_API_KEY=your_google_maps_api_key
```

## Summary

This notebook demonstrates a complete multi-agent AI application that:

1. **Ingests** mechanic business information
2. **Locates** the business using Google Maps integration
3. **Retrieves** street view imagery
4. **Edits** images to remove sensitive information
5. **Searches** the web for business details
6. **Writes** professional business bios
7. **Reviews** and refines the content
8. **Presents** results through an intuitive Gradio interface

### Key Technologies Used:
- **AWS Bedrock** for AI agent orchestration
- **Gradio** for the web interface
- **Google Maps API** for location services (simulated)
- **PIL/Pillow** for image processing
- **Multi-agent architecture** for task specialization

### Production Considerations:
- Implement actual Google Maps API integration
- Use real image editing APIs or ML models for phone number detection
- Add error handling and retry logic
- Implement caching for API calls
- Add authentication and rate limiting
- Store results in a database
- Implement monitoring and logging

### Next Steps:
1. Set up AWS credentials and Bedrock access
2. Obtain Google Maps API key
3. Deploy to AWS Lambda or EC2
4. Add more sophisticated image processing
5. Enhance bio generation with more data sources